# Streaming Twitter Data into a MySQL Database

In this tutorial session , I wanted to look at how we can use python and an API to stream data directly into a MySQL database.
We are going to be using the Twitter API to search for tweets containing specific keywords and stream this directly into our database. Once we have done this, the data will be available for further analysis at any time. This task requires a few things:
1. A Twitter account and API credentials
2. A MySQL database
3. The Tweepy and mysql-connector Python Libraries

# Twitter API
Before we access the API you need to set up a twitter app. I won’t do an in-depth tutorial on this but briefly, you need to do the following:
* go to the following website https://developer.twitter.com/ and create an account. (This step is a bit more involved then it used to be and involves providing a brief summary of what you intend to do with the tweets and what they will be used for. I believe it has something to do with new EU privacy laws.)
* Once you have verified your email you can log into your account. You should be able to create a new app on the following webpage: https://developer.twitter.com/en/apps
* Fill in all the details about your app and then create your access token.Make a note of your consumer key, consumer secret, OAuth access token and OAuth access token secret. These are needed to connect to the API.
For a more complete tutorial on this, I suggest this blog post. After these steps, our app is now able to connect to the Twitter streaming API provided we write the correct code. Next up, I will go through setting up the MySQL database so we have somewhere to store all of our data.

# MySQL Workbench

There are many different types of databases we could use for this particular task including NoSQL databases such as MongoDB or Redis. I have, however, chosen to use MySQL as it is still one of the most popular databases out there. Before we begin, we will need to install MySQL Workbench and MySQL server. [Here](https://www.youtube.com/watch?v=iOlJxOkp6sI) is a video tutorial explaining how to install both and set everything up to start collecting the data.

Once you have finished the tutorial above, you should have a connection and a schema/database set up (My database is imaginatively called twitterdb). After we have set up MySQL workbench and are somewhat familiar with the interface, we can finally create a table to store our twitter data. Creating a table is very straightforward and we can use the UI or even use queries. Using the UI we just right click on our database and click create a table. We can then input our column names and data types directly. At this point, it is worth thinking about the data we want to store and what kind of data types they will be. To get a better understanding of the data types we need we should take a peek at the TwitterAPI [documentation](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/intro-to-tweet-json). Essentially I want the **username** of the person who wrote the tweet, the **time** it was created, the **tweet**, the **retweet count**, the **place** the tweet originated and the **location** (more on these below). This corresponds to 6 columns plus the **primary key** and we can define the datatypes as follows:

* primary key: INT(11)
* username: VARCHAR(255)
* created_at: VARCHAR(45)
* tweet: TEXT
* retweet_count: INT(11)
* location: VARCHAR(100)
* place: VARCHAR(100)

# Python
OK now that we have our database set up, its’ time to jump into the Python code. There are a few things we want our code to do:
1. We want to create a class that allows us to connect to the Twitter API.
2. We also need to create some code that connects to our database and reads the data into the correct columns.
We are going to be using the Tweepy library which will make it very easy for us to connect to the API and start streaming the data. Before we start, we are again going to look at some delicious [documentation](https://tweepy.readthedocs.io/en/v3.5.0/). In the Tweepy documentation, we can find some really useful examples of the classes and methods we need to use to interact with the API. The code below is a simple example that allows us to connect to the API and print tweets from our timeline:

```
import tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)
```

Alright, so hopefully this is pretty straightforward. It looks like we just need to set our credentials, access the timeline and loop through it to print it out. What we want to do is a bit different. We want to stream live tweets into our database and according to the documentation, need to do the following three things:
* Create a class inheriting from StreamListener.
* Instantiate an object from this class.
* Use this object to connect to the API.

This seems straightforward enough. Lets see how we do this in Python (**Full code at end of post**). We need to import some libraries and also to set our tokens and secret keys and our password for the database. you can save all of these in the **setting.sh** file which gets called using the code below and sets the tokens and keys as environment variables.
```
subprocess.call(“./settings.sh”, shell=True)
```

Note that in our imports, we need **mysql-connector**. Again, here is some useful examples of how the library works. We can install any libraries that we don’t have using the pip command. We should then be able to import these libraries from our script.

Next up, we need to set up our class inheriting from StreamListener. We are going to give the class three methods. These are methods that the class already implements which we are going to override. The code below implements this.

Let’s go through this step by step to make sure everything is clear. The first method, on_connect() simply notifies us when we are connected to the stream. The on_error() method prints an error whenever our HTTP status code is not 200 (200 means everything worked). List of codes for those interested: https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

OK, the next method, on_data() is a little more complex. To understand this we will need to know a little bit more about the structure of the tweets. When we access the API we are getting a JSON response (very similar structure to a python dictionary). More info here.
Essentially our tweet object that is returned looks something like this:

```
{ “created_at”:”Thu Apr 06 15:24:15 +0000 2017", 
  “id”: 850006245121695744, 
  “id_str”: “850006245121695744”, 
  “text”: “1/ Today we’re sharing our vision for the future of the    Twitter API platform!nhttps://t.co/XweGngmxlP", 
  “user”: {}, 
  “entities”: {} }
```

So we have a JSON object which contains key, value pairs (note there are a few attributes not listed here that we will use). So what data do we actually want from this? There is actually quite a lot of information available from the tweet object and I recommend going through the documentation to see what attributes might interest you. For this analysis I chose to collect data on the username, the time the tweet was created (this is more useful if we collect tweets over time), the actual tweet, the country of the tweet, the location (which is more local) and finally the retweet count and this is reflected in the code above.

The final few lines call the **connect()** method which takes our variables as parameters. This code is all wrapped in a try, except statement to catch any errors, we may run into.

Alright, you may have noticed that we haven’t created the **connect()** method yet so let’s create it. This method is not surprisingly, going to connect to our database and feed in all the data. As I said before, the method takes in our variables created in the **on_data()** method of the StreamListener class as arguments and inserts them into the columns of the same name in our database. To connect to our database we simply use the **connector.connect** method and pass in our database info which we can find from MySQL workbench. If our connection was successful, a cursor object is created allowing us to execute SQL statements. 

Now we can write our query and insert the data into the correct table in our **twitterdb** database using the execute command. While is_connected() is true our database connection stays open and continually feeds the data into the database until we kill it in the terminal (using Ctrl+C).
We can create a list of words that we want to filter the stream for. I am a bit of a golf fan so I decided to search for words relating to golf. In practice, you can put whatever you want in this list.

Now we just need to set up our script to call these functions when we execute the file from the terminal. To access the API we need to pass our credentials as arguments to the **OAuthHandler** method and the **set_access_token** method. Next, we create the stream by passing in our verified api object and our listener. We can also create our list of words to **filter** for here. To start the stream we simply call the filter method on our stream object and pass in our list of words as an argument. I saved this script as **StreamSQL.py**.

If we want to run this code and start collecting tweets we can use the terminal. One important thing to note here is that we need to make sure our SQL server is up and running for the script to work so it is worth double checking this before we run the script.

We can open a terminal directly from the folder where we stored the script and simply type:
```
python StreamSQL.py
```

In [ ]:
import mysql.connector
from mysql.connector import Error
import tweepy
import json
from dateutil import parser
import time
import os
import subprocess

#importing file which sets env variable
subprocess.call("./settings.sh", shell = True)


consumer_key = os.environ['CONSUMER_KEY']
consumer_secret = os.environ['CONSUMER_SECRET']
access_token = os.environ['ACCESS_TOKEN']
access_token_secret = os.environ['ACCESS_TOKEN_SECRET']
password = os.environ['PASSWORD']


def connect(username, created_at, tweet, retweet_count, place , location):
    """
    connect to MySQL database and insert twitter data
    """
    try:
        con = mysql.connector.connect(host = 'localhost',
        database='twitterdb', user='root', password = password, charset = 'utf8')
        

        if con.is_connected():
            """
            Insert twitter data
            """
            cursor = con.cursor()
            # twitter, golf
            query = "INSERT INTO Golf (username, created_at, tweet, retweet_count,place, location) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(query, (username, created_at, tweet, retweet_count, place, location))
            con.commit()
            
            
    except Error as e:
        print(e)

    cursor.close()
    con.close()

    return


# Tweepy class to access Twitter API
class Streamlistener(tweepy.StreamListener):
    

    def on_connect(self):
        print("You are connected to the Twitter API")


    def on_error(self):
        if status_code != 200:
            print("error found")
            # returning false disconnects the stream
            return False

    """
    This method reads in tweet data as Json
    and extracts the data we want.
    """
    def on_data(self,data):
        
        try:
            raw_data = json.loads(data)

            if 'text' in raw_data:
                
                username = raw_data['user']['screen_name']
                created_at = parser.parse(raw_data['created_at'])
                tweet = raw_data['text']
                retweet_count = raw_data['retweet_count']

                if raw_data['place'] is not None:
                    place = raw_data['place']['country']
                    print(place)
                else:
                    place = None
                

                location = raw_data['user']['location']

                #insert data just collected into MySQL database
                connect(username, created_at, tweet, retweet_count, place, location)
                print("Tweet colleted at: {} ".format(str(created_at)))
        except Error as e:
            print(e)


if __name__== '__main__':

    # # #Allow user input
    # track = []
    # while True:

    # input1  = input("what do you want to collect tweets on?: ")
    # track.append(input1)

    # input2 = input("Do you wish to enter another word? y/n ")
    # if input2 == 'n' or input2 == 'N':
    #     break
    
    # print("You want to search for {}".format(track))
    # print("Initialising Connection to Twitter API....")
    # time.sleep(2)

    # authentification so we can access twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api =tweepy.API(auth, wait_on_rate_limit=True)

    # create instance of Streamlistener
    listener = Streamlistener(api = api)
    stream = tweepy.Stream(auth, listener = listener)

    track = ['golf', 'masters', 'reed', 'mcilroy', 'woods']
    #track = ['nba', 'cavs', 'celtics', 'basketball']
    # choose what we want to filter by
    stream.filter(track = track, languages = ['en'])